In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import numpy as np
import pandas as pd

from IPython.display import display

import CBOEFileParser
import VerticalSpreads
import CBOE_Downloader
import POP

import VerticalSpread

In [2]:
underlying_symbol = "BIDU"
risk_free_rate = 2.94

## POP Calculator - only needs to be ran at the beginning of the trading day

In [3]:
pop_calculator = POP.POP().get_ohlc_data(underlying_symbol)

# Update the Option Chain

In [4]:
# cboe_file = CBOE_Downloader.CBOE_Downloader(underlying_symbol).download().save_download().get_filename()
cboe_file = "/Users/glenn/Documents/CBOE_Data/BIDU_201807261013.dat"
# cboe_file = "/Users/glenn/Documents/CBOE_Data/GOOG_201807301217.dat"

option_chain_obj = CBOEFileParser.CBOEFileParser(
                    remove_options_after_days=60,
                    remove_options_before_days=30,
                    set_exchange=None) \
                    .parse_cboe_file(cboe_file=cboe_file) \
                    .add_greeks(risk_free_rate=0.298)

for expiration, calls, puts in option_chain_obj.items():
    print("Expiration: {}".format(expiration))
    print("Calls: {:,}".format(len(calls)))
    print("Puts: {:,}".format(len(puts)))
    
vertical_spreads = VerticalSpreads.VerticalSpreads() \
                                    .build_spreads(option_chain_obj) \
                                    .risk_limit(200)\
                                    .itm_spreads(option_chain_obj.underlying_spot) \
                                    .sky_view_spreads() \
                                    .add_pop(pop_calculator, option_chain_obj.underlying_spot) \
                                    .add_expected_return(pop_calculator, option_chain_obj.underlying_spot) \
                                    .write_to_mongo(mongo_address="mongodb://localhost:27017")
                
print("Current Spot: {}".format(option_chain_obj.underlying_spot))
for expiration, bull_calls, bull_puts, bear_calls, bear_puts in vertical_spreads.items():
    print("Expiration: {}".format(expiration))
    print("Bull Puts: {:,}".format(len(bull_puts)))
    print("Bear Calls: {:,}".format(len(bear_calls)))

Expiration: 2018-08-31 23:59:00
Calls: 47
Puts: 47
Expiration: 2018-09-07 23:59:00
Calls: 29
Puts: 29
Expiration: 2018-09-21 23:59:00
Calls: 35
Puts: 29


In [31]:
vertical_spreads = VerticalSpreads.VerticalSpreads() \
                                    .build_spreads(option_chain_obj) \
                                    .risk_limit(200)\
                                    .itm_spreads(option_chain_obj.underlying_spot) \
                                    .sky_view_spreads() \
                                    .add_pop(pop_calculator, option_chain_obj.underlying_spot) \
                                    .add_expected_return(pop_calculator, option_chain_obj.underlying_spot) \
                                    .write_to_mongo(mongo_address="mongodb://localhost:27017")
                
print("Current Spot: {}".format(option_chain_obj.underlying_spot))
for expiration, bull_calls, bull_puts, bear_calls, bear_puts in vertical_spreads.items():
    print("Expiration: {}".format(expiration))
    print("Bull Puts: {:,}".format(len(bull_puts)))
    print("Bear Calls: {:,}".format(len(bear_calls)))


Current Spot: 259.54
Expiration: 2018-08-31 23:59:00
Bull Puts: 4
Bear Calls: 5
Expiration: 2018-09-07 23:59:00
Bull Puts: 0
Bear Calls: 0
Expiration: 2018-09-21 23:59:00
Bull Puts: 0
Bear Calls: 0
